# Pessimistic and Optimistic Bound on the Robustness Radius

In this notebook we try to study the different bounds of the robustness radius given by several methods (lip certificate, adv attacks, CROWN).

In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.model import Sequential
from deel.lip.activations import GroupSort
from deel.lip.losses import MulticlassHKR, MulticlassKR
from keras.layers import Input, Flatten
from keras.optimizers import Adam
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
import numpy as np
import keras.ops as K
import matplotlib.pyplot as plt

In [17]:
import pandas as pd

df = pd.read_pickle("/home/aws_install/robustess_project/lip_notebooks/data/Radius_Data/Radius_FMNIST.pkl")


In [18]:
df.tail()

,Index,Label_GT,Predicted_Label,Lipschitz_Constant,Robust_Epsilon,Adv_Epsilon_AA,Adv_Epsilon_PGD
5,6,0,0,1.0,0.30577415,1.4137657,1.7910519
6,7,0,0,1.0,0.06562792,0.31114691,0.43479022
7,8,0,0,1.0,0.3217299,1.6402177,1.9918573
8,9,0,0,1.0,0.041682042,0.2284363,0.38707727
9,10,0,0,1.0,0.5669894,2.728335,3.3219976


### Load Fashion MNIST Dataset

the dataset has ten classes.

In [3]:
# load data
(x_train, y_train_ord), (x_test, y_test_ord) = fashion_mnist.load_data()
# standardize and reshape the data
x_train = np.expand_dims(x_train, -1) / 255
x_test = np.expand_dims(x_test, -1) / 255
# one hot encode the labels
y_train = to_categorical(y_train_ord)
y_test = to_categorical(y_test_ord)

In [4]:
x_train = np.transpose(x_train,(0,3,1,2))
x_test = np.transpose(x_test,(0,3,1,2))

In [5]:
np.min(x_train)

np.float64(0.0)

In [5]:
# model = keras.models.load_model("/home/aws_install/robustess_project/deel-lip/docs/notebooks/demo4_vanilla_fashionMNIST_channelfirst.keras")
# model.compile(
#     # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
#     # note also in the case of lipschitz networks, more robustness require more parameters.
#     loss=MulticlassHKR(alpha=100, min_margin=0.25),
#     optimizer=Adam(1e-4),
#     metrics=["accuracy", MulticlassKR()],)
# model.summary()

In [6]:
vanilla_model = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo4_vanilla_fashionMNIST_channelfirst_False_disj_Neurons.keras")
vanilla_model.compile(
    # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
    # note also in the case of lipschitz networks, more robustness require more parameters.
    loss=MulticlassHKR(alpha=100, min_margin=0.25),
    optimizer=Adam(1e-4),
    metrics=["accuracy", MulticlassKR()],)
vanilla_model.summary()

Model: "hkr_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spectral_conv2d (Conv2D)        │ (None, 16, 28, 28)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ scaled_l2_norm_pooling2d        │ (None, 16, 14, 14)     │             0 │
│ (ScaledL2NormPooling2D)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_conv2d_1 (Conv2D)      │ (None, 32, 14, 14)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ scaled_l2_norm_pooling2d_1      │ (None, 32, 7, 7)       │             0 │
│ (ScaledL2NormPooling2D)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1568)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (Dense)          │ (None, 64)             │       100,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1 (Dense)        │ (None, 10)             │           640 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 105,856 (413.50 KB)

 Trainable params: 105,856 (413.50 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# strategy: first
# we select a sample from each class.
images_list = []
labels_list = []
idx_list = []
# select only a few element from the test set
# selected = np.random.choice(len(y_test_ord), 500)
sub_y_test_ord = y_test_ord[:400]
sub_x_test = x_test[:400]
# drop misclassified elements
misclassified_mask = K.equal(
    K.argmax(vanilla_model.predict(sub_x_test), axis=-1), sub_y_test_ord
)
misclassified_mask = misclassified_mask.detach().cpu().numpy()

idx_global = np.arange(len(sub_x_test))[misclassified_mask]
sub_x_test = sub_x_test[misclassified_mask]
sub_y_test_ord = sub_y_test_ord[misclassified_mask]


for i in range(10):
    # select the 20 firsts elements of the ith label
    label_mask = sub_y_test_ord == i
    x = sub_x_test[label_mask][:20]
    y = sub_y_test_ord[label_mask][:20]
    idx = idx_global[label_mask][:20]

    # convert it to tensor for use with foolbox
    images = K.convert_to_tensor(x.astype("float32"), dtype="float32")
    labels = K.convert_to_tensor(y, dtype="int64")
    # repeat the input 10 times, one per misclassification target
    for j in range(20):
        images_list.append(images[j])
        labels_list.append(labels[j])
        idx_list.append(idx[j])
images = K.convert_to_tensor(images_list)
labels = K.convert_to_tensor(labels_list)


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


images shape = (nombre classes, nbr de points , channels, dim img 1, dim img 2)

### Get the Pessimistic Radius via Lip Constraints (l2 norm)

In [12]:
def compute_certificate(images, model, L=1):    
    values, _ = K.top_k(model(images), k=2)
    certificates = (values[:, 0] - values[:, 1]) / (np.sqrt(2)*L)
    return certificates.detach().cpu().numpy()   

In [13]:
lip_radius = compute_certificate(images, vanilla_model)

In [14]:
lip_radius[:10]

array([0.3496229 , 0.13969153, 0.1685597 , 0.18583024, 0.3157398 ,
       0.30577415, 0.06562792, 0.3217299 , 0.04168204, 0.5669894 ],
      dtype=float32)

### Get Optimistic Radius via AutoAttacks

In [9]:
import keras.ops as K
import matplotlib.pyplot as plt
import torchattacks
import torch
import torch.nn as nn
import torchattacks
from robustbench.utils import clean_accuracy

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
def starting_point_dichotomy(idx, images, targets):
    mask_different_classes = targets[idx] != targets
    images_diffferent_classes = images[mask_different_classes]
    return K.amin((images[idx] - images_diffferent_classes).square().sum(dim=(1, 2, 3)).sqrt())

In [46]:
starting_point_dichotomy(1, images, labels)


tensor(4.8500, device='cuda:0')

In [115]:
def single_compute_optimistic_radius_PGD(idx, images, targets, certificates, model, n_iter = 10):
    image = images[idx:idx+1]
    target = targets[idx:idx+1]
    certificate = certificates[idx:idx+1]
    # We use dichotomy algorithm to fine the smallest optimistic radius
    # We start from the closest point with different class
    eps_working = d_up = starting_point_dichotomy(1, images, targets)
    d_low = K.convert_to_tensor(certificate)
    for _ in range(n_iter):
        print(eps_working)
        eps_current = (d_up+d_low)/2
        # atk_van = torchattacks.PGDL2(model, eps=eps_current, alpha=eps_current/5, steps=10, random_start=True)
        atk_van = torchattacks.PGDL2(model, eps=eps_current, alpha=eps_current/5, steps=int((10*eps_current)), random_start=True)
        adv_image = atk_van(image, target)
        # return 0 if the attack doesn't work
        if (K.argmax(model(adv_image), axis=1) == target):
            d_low = eps_current
        else:
            eps_working = d_up = (image - adv_image).square().sum(dim=(1, 2, 3)).sqrt()
    return eps_working


In [116]:
def single_compute_optimistic_radius_AA(idx, images, targets, certificates, model, n_iter = 10):
    image = images[idx:idx+1]
    target = targets[idx:idx+1]
    certificate = certificates[idx:idx+1]
    # We use dichotomy algorithm to fine the smallest optimistic radius
    # We start from the closest point with different class
    eps_working = d_up = starting_point_dichotomy(1, images, targets)
    d_low = d_low = K.convert_to_tensor(certificate)
    for _ in range(n_iter):
        print(eps_working)
        eps_current = (d_up+d_low)/2
        atk = torchattacks.AutoAttack(model, norm='L2', eps=eps_current)
        adv_image = atk(image, target)
        if (K.argmax(model(adv_image), axis=1) == target):
            d_low = eps_current
        else:
            eps_working = d_up = (image - adv_image).square().sum(dim=(1, 2, 3)).sqrt()
    return eps_working

In [117]:
i = 0
eps_current = 1.9

In [118]:
single_compute_optimistic_radius_PGD(i, images, labels, lip_radius, vanilla_model, n_iter = 10)

tensor(4.8500, device='cuda:0')
tensor([2.5952], device='cuda:0')
tensor([2.5952], device='cuda:0')
tensor([2.0312], device='cuda:0')
tensor([2.0312], device='cuda:0')
tensor([1.8895], device='cuda:0')
tensor([1.8895], device='cuda:0')
tensor([1.8531], device='cuda:0')
tensor([1.8531], device='cuda:0')
tensor([1.8531], device='cuda:0')


tensor([1.8531], device='cuda:0')

In [111]:
single_compute_optimistic_radius_AA(i, images, labels, lip_radius, vanilla_model, n_iter = 10)

tensor(4.8500, device='cuda:0')
tensor([2.5669], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([2.5669], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([1.9756], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([1.7046], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([1.5707], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([1.5707], device='cuda:0', grad_fn=<SqrtBackward0>)


KeyboardInterrupt: 

In [381]:
print(lip_radius[i:i+1])

[0.3496229]


In [382]:
atk_van = torchattacks.PGDL2(vanilla_model, eps=eps_current, alpha=3*eps_current/256, steps=int((eps_current)*256), random_start=True)
adv_image = atk_van(images[i:i+1], labels[i:i+1])
print((images[i:i+1] - adv_image).square().sum(dim=(1, 2, 3)).sqrt() if (K.argmax(vanilla_model(adv_image), axis=1) != labels[i:i+1]) else 0)

tensor([1.9000], device='cuda:0')


In [383]:
atk_van = torchattacks.PGDL2(vanilla_model, eps=eps_current, alpha=eps_current/5, steps=int((10*eps_current)), random_start=True)
adv_image = atk_van(images[i:i+1], labels[i:i+1])
print((images[i:i+1] - adv_image).square().sum(dim=(1, 2, 3)).sqrt() if (K.argmax(vanilla_model(adv_image), axis=1) != labels[i:i+1]) else 0)

tensor([1.8979], device='cuda:0')


In [379]:
atk = torchattacks.AutoAttack(vanilla_model, norm='L2', eps=eps_current)
adv_image = atk(images[i:i+1], labels[i:i+1])
print((images[i:i+1] - adv_image).square().sum(dim=(1, 2, 3)).sqrt())

tensor([1.7852], device='cuda:0', grad_fn=<SqrtBackward0>)


We found an adversarial attack for eps= tensor([2.0949], device='cuda:0')
Launching Dichotomy :
0 1.0488687 False
1 1.573303 False
2 1.8355201 False
3 1.9666288 True
4 1.9010744 False
5 1.9338516 False
6 1.9502401 True
7 1.9420459 True
8 1.9379487 False
9 1.9399973 True


np.float32(1.9399973)

In [33]:
eps_PGD = []
eps_AA = []

In [13]:
for i in range(images.shape[0]):
    eps_AA.append(single_compute_optimistic_radius_AA(images[i:i+1], labels[i:i+1], lip_radius[i:i+1], vanilla_model, n_iter = 10))

KeyboardInterrupt: 

In [14]:
for i in range(3):
    eps_PGD.append(single_compute_optimistic_radius_PGD(images[i:i+1], labels[i:i+1], lip_radius[i:i+1], vanilla_model, n_iter = 10))

We found an adversarial attack for eps= [2.3915052]
Launching Dichotomy :
0 1.1957526 False
1 1.7936289 False
2 2.092567 True
3 1.943098 False
4 2.0178325 True
5 1.9804652 False
6 1.9991488 False
7 2.0084906 False
8 2.0131617 True
9 2.010826 False
10 2.011994 False
11 2.0125778 False
12 2.0128698 False
13 2.0130157 True
We found an adversarial attack for eps= [0.47378108]
Launching Dichotomy :
0 0.23689054 False
1 0.3553358 False
2 0.41455844 False
3 0.44416976 False
4 0.45897543 False
5 0.46637827 True
6 0.46267685 True
7 0.46082616 False
8 0.46175152 True
9 0.46128884 True
We found an adversarial attack for eps= [1.2187829]
Launching Dichotomy :
0 0.60939145 False
1 0.9140872 False
2 1.0664351 False
3 1.142609 False
4 1.180696 False
5 1.1997395 True
6 1.1902177 False
7 1.1949786 True
8 1.1925981 False
9 1.1937883 True


In [15]:
eps_PGD

[np.float32(2.0130157), np.float32(0.46128884), np.float32(1.1937883)]

In [19]:
# eps_AA = K.convert_to_tensor(eps_AA)
eps_PGD = K.convert_to_tensor(eps_PGD)

In [74]:
labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9, 9, 9, 9, 9, 9, 9], device='cuda:0')

In [ ]:
# # Print results
# print("Image #     Certificate     Distance to adversarial")
# print("--------------------------------------------------------------------------------------------------------------")
# for i in range(10):
#     print(f"Image {i}        {lip_radius[i]:.3f}                {eps_AA[i]:.2f}                    {eps_PGD[i]:.2f}  ")

### Generating Dataframe

In [77]:
type(lip_radius)

numpy.ndarray

In [17]:
import pandas as pd

In [20]:
# penser à tout convertir en numpy
total_points = images.shape[0]

# Création du DataFrame avec une colonne d'index de 1 à 200
df = pd.DataFrame({
    # 'Index': np.arange(1, total_points + 1),
    # 'Label_GT': labels.detach().cpu().numpy(),  
    # 'Label_Predit': np.argmax(vanilla_model(images).detach().cpu().numpy(), axis=1),  
    # 'Constante_Lipschitz': np.ones(total_points), 
    # 'Epsilon_Robuste': lip_radius,
    # 'Epsilon_Adv_AA': eps_AA,
    'Epsilon_Adv_PGD': eps_PGD
})

# Affichage des premières lignes du DataFrame
print(df.head())

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.